In [13]:
from numpy import dtype
from kdg import kdf, kdn, kdcnn
from kdg.utils import get_ece, plot_reliability
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Activation, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization
from tensorflow.keras import activations
from tensorflow.keras import backend as bknd
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os 
import numpy as np
import openml
from sklearn.metrics import cohen_kappa_score
from kdg.utils import get_ece
import numpy as np
from scipy.stats import multivariate_normal
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split


In [41]:
def getLeNet(input_shape, num_classes):
    model = keras.Sequential()
    inputs = Input(shape=input_shape)
    x = Conv2D(6, (3,3), kernel_initializer='he_uniform')(inputs)
    x = Activation('relu')(x)
    x = AveragePooling2D()(x)
    x = Conv2D(16, (3,3), kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((2,2))(x)

    x = Flatten()(x)
    x = Dense(120, kernel_initializer='he_uniform')(x)
    x = Activation('relu')(x)
    x = Dense(84, kernel_initializer='he_uniform')(x)
    x = Activation('relu')(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model
    

In [63]:
### Hyperparameters ###
subtract_pixel_mean = True
normalize = True
classes_to_consider = [[0,1], [2,3],
                       [4,5], [6,7],
                       [8,9]]
seeds = [0,100,200,300,400]

compile_kwargs = {
        "loss": "binary_crossentropy",
        "optimizer": keras.optimizers.Adam(3e-4),
    }
callback = keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=True)
fit_kwargs = {
        "epochs": 100,
        "batch_size": 32,
        "verbose": False,
        "callbacks": [callback],
    }

In [66]:
### preprocess the data ###

def get_data(classes):
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
    input_shape = x_train.shape[1:]

    train_idx = np.where(y_train==classes_to_consider[0])[0]
    test_idx = np.where(y_test==classes_to_consider[0])[0]

    for ii in classes[1:]:
        train_idx = np.concatenate((
                        train_idx,
                        np.where(y_train==ii)[0]
                    ))
        test_idx = np.concatenate((
                        test_idx,
                        np.where(y_test==ii)[0]
                    ))

    x_train, y_train = x_train[train_idx], y_train[train_idx]
    x_test, y_test = x_test[test_idx], y_test[test_idx]
    
    _, y_train = np.unique(y_train, return_inverse=True)
    _, y_test = np.unique(y_test, return_inverse=True)
    
    if normalize:
        x_train = x_train.astype('float32') / 255
        x_test = x_test.astype('float32') / 255

    if subtract_pixel_mean:
        x_train_mean = np.mean(x_train, axis=0)
        x_train -= x_train_mean
        x_test -= x_train_mean
    return (x_train, y_train), (x_test, y_test)

In [67]:
### train LeNet5 ###

for task, classes in enumerate(classes_to_consider):
    (x_train, y_train), (x_test, y_test) = get_data(classes)
    for seed in seeds:
        np.random.seed(seed)
        nn = getLeNet(input_shape, num_classes=len(np.unique(y_train)))
        nn.compile(**compile_kwargs)
        history = nn.fit(x_train, keras.utils.to_categorical(y_train), **fit_kwargs)
        
        y_pred = np.argmax(nn.predict(x_test), axis=1)
        print('Trained model with classes ', classes, ' seed ', seed)
        print('Accuracy:', np.mean(y_pred==y_test.reshape(-1)))
        nn.save('lenet_models/cifar10_model_'+str(task)+'_'+str(seed))

2023-08-04 16:50:01.536062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 35: early stopping
50/94 [==============>...............] - ETA: 0s

2023-08-04 16:51:50.442372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 0s 2ms/step
Trained model with classes  [0, 1]  seed  0
Accuracy: 0.9253333333333333


INFO:tensorflow:Assets written to: lenet_models/cifar10_model_0_0/assets


...

Epoch 72: early stopping
51/94 [===============>..............] - ETA: 0s

2023-08-04 18:14:40.764616: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - 0s 2ms/step
Trained model with classes  [8, 9]  seed  400
Accuracy: 0.81


INFO:tensorflow:Assets written to: lenet_models/cifar10_model_4_400/assets


INFO:tensorflow:Assets written to: lenet_models/cifar10_model_4_400/assets


In [ ]:
### load models ###
nn = keras.models.load_model('lenet_models/cifar10_model_0_0')
